## US Trade Data Extractor

This module provides functions to extract international trade data of imports and exports from the US Census website and their provided API.

### Dependencies:

- `requests`: *A library for making HTTP requests.*
- `pandas`: *A library for data manipulation and analysis.*
- `numpy`: *A library for numerical computations.*
- `bs4` from `beautifulsoup4`: *A library for web scraping.*
- `io`: *A library for handling I/O operations.*
- `dataclasses`: *A library for creating data classes.*


['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

In [4]:
import requests
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from io import StringIO
from dataclasses import dataclass
from pathlib import Path
import zipfile
import os

## Module Installation Checker

This function verifies the presence of a specified Python module and installs it if it's not already available. It takes one parameter:

 - *module_name: The name of the Python module to check and install if needed.*

The function returns True if the module is either already installed or successfully installed. It returns False if the installation fails.


In [5]:
import importlib.util
import subprocess

def check_and_install_module(module_name):
    try:
        exists = importlib.util.find_spec(module_name)
        if exists is None:
            raise ImportError
        print(f"{module_name} module is already installed.")
        return True

    except ImportError:
        print(f"{module_name} module is not installed. Attempting to install...")

        try:
            subprocess.check_call(["pip", "install", module_name])
            print(f"{module_name} module installed successfully.")
            return True

        except subprocess.CalledProcessError:
            print(f"Failed to install {module_name} module. Please install it manually.")
            return False


## Data Class

This data class represents information related to accessing US Census data API.

- `host` (str): *The base URL of the Census data API.*
- `dataHost` (str): *The specific data category or endpoint within the API (time series data for international trade in our case)*
- `api` (str): *The API key for accessing the Census data.*


In [6]:
@dataclass
class data:
    host: str 
    dataHost: str 
    api: str


d = data(
            host = "https://api.census.gov/data",
            dataHost= "timeseries/intltrade",
            api = "919d92868c70f630a9eda2dc7445e4ad1f614ab6"
         )


## URL Builder Class

This class provides functionality to build URLs for available datasets on the fly,
based on options selected by user, for accessing US Census data API endpoints.

### Methods:

- `__init__(self, host, data_host)`: *Initializes the URL builder with the base host and data host.*
- `build_url(self)`: *Builds the URL based on user input for data type (whether imports or exports) and classification system.*
- `_build_url_internal(self, ImportExport, ENDPOINT)`: *Builds the URL internally based on provided parameters.*




### Available datasets through different classification systems are as follows, Monthly U.S. Imports / Exports by: 

| Data set                                 |
|------------------------------------------|
| Harmonized System (HS)                   |
| Harmonized System (HS) by State          |
| Harmonized System (HS) by Port           |
| North American Industry Classification  |
| NAICS by State                           |
| End-use                                  |
| Standard International Trade Classification (SITC) |
| Advance Technology (Hi-tech)             |
| USDA (Ag or NonAg)                       |


In [7]:
class UrlBuilder:
    """
    URL Builder Class

    Provides functionality to build URLs for accessing US Census data API endpoints.
    """

    def __init__(self, host, data_host):
        """
        Initializes the URL builder with the base host and data host.

        Parameters:
            host (str): The base URL of the Census data API.
            data_host (str): The specific data category or endpoint within the API.
        """
        self.host = host
        self.data_host = data_host
    
    def build_url(self):
        """
        Builds the URL based on user input for data type and classification system.

        Returns:
            str: The constructed URL.
        """
        ImportExport = "imports" if int(input("Do you want imports (1) or exports(0) data?")) else "exports"

        Options = f"""
            You chose {ImportExport}, which classification would you like to use further? \n

            Available datasets through different classification systems are as follows, Monthly U.S. Imports / Exports by: 
            [Choose the number corresponding to options below (for now just choose 1)] 

            1. HS Code
            2. NAICS Code 
            3. End-use Code
            4. SITC Code
            5. USDA
            6. HiTech Code
            7. HS by State
            8. NAICS by State
            9. HS by Port
        """
        print(Options)
        
        SelectedOption = int(input(Options))
        Endpoints = {1: "hs",
                     2: "naics",
                     3: "enduse",
                     4: "sitc",
                     5: "usda",
                     6: "hitech",
                     7: "statehs",
                     8: "statenaics",
                     9: "porths"}

        ENDPOINT = Endpoints.get(SelectedOption)

        return self._build_url_internal(ImportExport, ENDPOINT)

    def _build_url_internal(self, ImportExport, ENDPOINT):
        """
        Builds the URL internally based on provided parameters.

        Parameters:
            ImportExport (str): Indicates whether the data is for imports or exports.
            ENDPOINT (str): The specific classification system endpoint.

        Returns:
            str: The constructed URL.
        """
        BASE_URL_COMPONENTS = [self.host, self.data_host, ImportExport, ENDPOINT]
        baseURL = "/".join(BASE_URL_COMPONENTS)
        return baseURL


## Data Fetcher Class

This class provides static methods to fetch data variables (like date, values (in USD), Product Quantities) from a given URL.

### Methods:

- `get_variables(url)`: *Static method to fetch variables from the provided URL.*
- `_html_table_to_dataframe(HTML_CONTENT)`: *Static method to convert HTML content (Variables that we fetch) to a pandas DataFrame.*

In [8]:
class DataFetcher:
    """
    Data Fetcher Class

    Provides static methods to fetch data variables from a given URL.
    """

    @staticmethod
    def get_variables(url):
        """
        Static method to fetch variables from the provided URL.

        Parameters:
            url (str): The URL to fetch data variables from.

        Returns:
            pd.DataFrame: A pandas DataFrame containing the fetched data variables, or None if the fetch fails.
        """
        VARIABLES_URL = "/".join([url, "variables.html"]) 
        response = requests.get(VARIABLES_URL)
        if response.status_code == 200:
            return DataFetcher._html_table_to_dataframe(response.text)
        else:
            print(f"Failed to fetch HTML. Status code: {response.status_code}")
            return None
    
    @staticmethod
    def _html_table_to_dataframe(HTML_CONTENT):
        """
        Static method to convert HTML content to a pandas DataFrame.

        Parameters:
            HTML_CONTENT (str): The HTML content to be converted.

        Returns:
            pd.DataFrame: A pandas DataFrame containing the parsed HTML table, or None if no tables are found.
        """
        soup = BeautifulSoup(HTML_CONTENT, 'html.parser') # Parse HTML content using BeautifulSoup
        html_string = str(soup) # Convert the HTML content to a string
        html_io = StringIO(html_string) # Use StringIO to wrap the HTML string
        tables = pd.read_html(html_io) # Extract tables from HTML using read_html() method
        if not tables:
            return None # If no tables found, return None
        df = tables[0] # There is only one table and that is what we want!
        return df[:-1]



## Data Retriever Class

This class provides a static method to retrieve data from a given URL with specified predicates.

### Methods:

- `retrieve_data(url, predicates)`: Static method to retrieve data from the provided URL with specified predicates.



In [9]:

class DataRetriever:
    """
    Data Retriever Class

    Provides a static method to retrieve data from a given URL with specified predicates.
    """

    @staticmethod
    def retrieve_data(url, predicates):
        """
        Static method to retrieve data from the provided URL with specified predicates.

        Parameters:
            url (str): The URL to retrieve data from.
            predicates (dict): The predicates to be included in the request.

        Returns:
            pd.DataFrame: A pandas DataFrame containing the retrieved data, or None if an error occurs.
        """
        RESULTS = requests.get(url, params=predicates)

        if RESULTS.status_code == 200:
            DATA = RESULTS.json()
            DF = pd.DataFrame(columns=DATA[0], data=DATA[1:])
        else:
            print("Error:", RESULTS.status_code)
            DF = None
    
        return DF, RESULTS


## DataHandler Class

This class provides methods for writing and reading data from CSV files.



#### write_to_csv(dataframe, filename)
- **Description**: Writes a DataFrame to a CSV file.
- **Parameters**: `dataframe` to write and `filename` to write to
- **Returns**: None

#### read_from_csv(filename)
- **Description**: Reads data from a CSV file and returns it as a DataFrame.
- **Parameters**: `filename` to read
- **Returns**: If the file exists, pandas DataFrame containing the data from CSV or None








In [10]:
class DataHandler:
    @staticmethod
    def to_parquet(dataframe, parquet_filename):
        # Create 'data' directory if it doesn't exist
        data_dir = os.path.join(os.getcwd(), 'data')
        os.makedirs(data_dir, exist_ok=True)

        # Writing dataframe to Parquet file within 'data' directory
        parquet_path = os.path.join(data_dir, parquet_filename)
        dataframe.to_parquet(parquet_path, index=False)

    @staticmethod
    def from_parquet(parquet_filename):
        # Reading Parquet file into a DataFrame
        parquet_path = os.path.join(os.getcwd(), 'data', parquet_filename)
        dataframe = pd.read_parquet(parquet_path)
        return dataframe




## Usage Example

This example demonstrates how to use the `UrlBuilder` and `DataFetcher` classes to retrieve data variables from the US Census website.


In [13]:
# Initialize URL Builder
urlBuilder = UrlBuilder(d.host, d.dataHost)

# Build URL
importsUrl = urlBuilder.build_url()
exportsUrl = urlBuilder.build_url()

# Get Variables
importsVarsDF = DataFetcher.get_variables(url=importsUrl)
exportsVarsDF = DataFetcher.get_variables(url=exportsUrl)

# Instantiate the retriever and handler classes
dr = DataRetriever()
dh = DataHandler()





            You chose imports, which classification would you like to use further? 


            Available datasets through different classification systems are as follows, Monthly U.S. Imports / Exports by: 
            [Choose the number corresponding to options below (for now just choose 1)] 

            1. HS Code
            2. NAICS Code 
            3. End-use Code
            4. SITC Code
            5. USDA
            6. HiTech Code
            7. HS by State
            8. NAICS by State
            9. HS by Port
        

            You chose exports, which classification would you like to use further? 


            Available datasets through different classification systems are as follows, Monthly U.S. Imports / Exports by: 
            [Choose the number corresponding to options below (for now just choose 1)] 

            1. HS Code
            2. NAICS Code 
            3. End-use Code
            4. SITC Code
            5. USDA
            6. HiTech Code
        

In [14]:
importsUrl

'https://api.census.gov/data/timeseries/intltrade/imports/hs'

<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8">
<meta name="viewport" content="width=device-width, initial-scale=1.0">
<title>Variables for retrieval of imports data</title>
<style>
    table {
        width: 60%;
        border-collapse: collapse;
    }
    th, td {
        padding: 8px;
        text-align: left;
        border-bottom: 1px solid #ddd;
    }
    th {
        background-color: #f2f2f2;
    }
            td {
    max-width: 400px; /* Adjust the maximum width as needed */
    overflow: auto; /* Add a scrollbar for overflow */
}
</style>
</head>
<body>

<h2>Variables for retrieval of imports data</h2>
<h3>Helpful to understand the data we're trying to pull</h3>

<table>
    <thead>
        <tr>
            <th>Variable Name</th>
            <th>Context</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td><strong>I_COMMODITY</strong></td>
            <td>
                <p>Import Commodity or HS Code in the following case (Could be 2, 4, 6 or 10 character depending on Commodity Level)</p>
                <ul>
                    <li>
                        If it is 2 characters, it determines the chapter of commodity, which is a broad classification
                            <strong><em>for example code 27 includes includes everything in MINERAL FUEL, OIL ETC.; BITUMIN SUBST; MINERAL WAX </em></strong>
                    </li> </br>
                    <li>
                        If it is 4 characters, than it is further classification of all goods in chapter 27, denoted by first 2 characters and headings in the next 2 characters 
                            <strong><em>for example code 2701 includes includes everything in COAL; BRIQUETTES, OVOIDS ETC. MFR FROM COAL, which is first heading of chapter 27 </em></strong>
                    </li></br>
                    <li>
                        If it is 6 characters, than it is further classification of all goods in chapter 27, denoted by first 2 characters and a specific heading in the following 2 characters and a sub heading of the commodity in the last two characters
                        <strong><em>for example code 270111 includes includes everything in ANTHRACITE COAL, NOT AGGLOMERATED,  which is eleventh subheading of first heading of chapter 27</em></strong>
                    </li></br>
                    <li>
                        If it is 10 characters, than it is the exact identity or code of the product being imported
                        <strong><em>for example code 2701110010 is the product METALLURGICAL BITUMINOUS COAL NOT AGGLOMERATED, which is tenth product in eleventh subheading in the first heading of chapter 27 </em></strong>
                    </li></br>
                    <li>
                        To understand more in depth, please refer the following <a href="https://www.freightos.com/freight-resources/harmonized-system-code-finder-hs-code-lookup/">link</a> for easy explanation
                    </li>
                </ul></br></br>
            </td>
        </tr>
        <tr>
            <td><strong>I_COMMODITY_SDESC</strong></td>
            <td>
                <p>A short description of the Commodity Codes</p>
                <ul>
                    <li> <strong><em> 27 </em></strong> , comes under  <strong><em> HS2 COMM_LVL: </em></strong> with short description as MINERAL FUEL, OIL ETC.; BITUMIN SUBST; MINERAL WAX</li>
                    <li> <strong><em> 2701 </em></strong> , comes under  <strong><em> HS4 COMM_LVL: </em></strong> with short description as COAL; BRIQUETTES, OVOIDS ETC. MFR FROM COAL</li>
                    <li> <strong><em> 270111 </em></strong> , comes under  <strong><em> HS6 COMM_LVL: </em></strong> with short description as ANTHRACITE COAL, NOT AGGLOMERATED</li>
                    <li> <strong><em> 2701120010 </em></strong> , comes under  <strong><em> HS10 COMM_LVL: </em></strong> with short description as METALLURGICAL BITUMINOUS COAL NOT AGGLOMERATED</li>
                </ul></br></br>
            </td>
        </tr>
        <tr>
            <td><strong>COMM_LVL</strong></td>
            <td>
                <p>Indicates whether a row contains information of a commodity on broad level or detailed level (HS2, HS4, HS6 or HS10)</p></br></br>
            </td>
        </tr>
        <tr>
            <td><strong>DISTRICT</strong></td>
            <td>Code of the district in which the goods were imported</br></br></td>
        </tr>
        <tr>
            <td><strong>DIST_NAME</strong></td>
            <td>Name of the district in which the goods were imported</br></br></td>
        </tr>
        <tr>
            <td><strong>CTY_CODE</strong></td>
            <td>Code of the country from where goods were imported into the states</br></br></td>
        </tr>
        <tr>
            <td><strong>CTY_NAME</strong></td>
            <td>Name of the country from where goods were imported into the states</br></br></td>
        </tr>
        <tr>
            <td><strong>GEN_VAL_MO</strong></td>
            <td>General Imports Value per month (measures the total physical arrivals of merchandise from foreign countries)</br></br></td>
        </tr>
        <tr>
            <td><strong>GEN_VAL_YR</strong></td>
            <td>General Imports Value year to date (We won't need this, we can implement rolling or expanding windows ourselves rather than putting load on the APIs)</br></br></td>
        </tr>
        <tr>
            <td><strong>CON_VAL_MO</strong></td>
            <td>Consumption Value of Imports per month (measures the total of merchandise that has physically cleared through Customs immediately or after withdrawal for consumption from Custody)</br></br></td>
        </tr>
        <tr>
            <td><strong>CON_VAL_YR</strong></td>
            <td>Consumption Value of Imports year to date (Again We won't need this, we can implement rolling or expanding windows ourselves rather than putting load on the APIs)</br></br></td>
        </tr>
        <tr>
            <td><strong>SUMMARY_LVL</strong></td>
            <td>
                SUMMARY_LVL has only two potential values: CGP and DET. We choose DET for our Analysis. CGP represents data aggregated by different country groupings, such as geographic regions (e.g. Africa), international organizations (e.g. European Union), or free trade agreements (e.g. NAFTA). DET represents individual country data not summarized by any country groupings. The SUMMARY_LVL parameter is useful if you want to only show trade data by individual countries in your application (using DET) or if you want to exclude individual countries and only show trade by country groupings (using CGP).</br></br>
            </td>
        </tr>
        <tr>
            <td><strong>SUMMARY_LVL2</strong></td>
            <td>
                The second parameter that accounts for different levels of data summarization is SUMMARY_LVL2. SUMMARY_LVL2 allows you to view and pull data from the API based on which variables from the trade datasets are combined and summarized (Kind of pre-performed Group By Tasks like we do in Pandas) </br></br></br>
                For Example, if a row contains values as follows: </br></br>
                    <table border="1">
                        <thead>
                            <tr>
                                <th></th>
                                <th>I_COMMODITY_SDESC</th>
                                <th>COMM_LVL</th>
                                <th>DISTRICT</th>
                                <th>DIST_NAME</th>
                                <th>CTY_CODE</th>
                                <th>CTY_NAME</th>
                                <th>GEN_VAL_MO</th>
                                <th>CON_VAL_MO</th>
                                <th>YEAR</th>
                                <th>MONTH</th>
                                <th>I_COMMODITY</th>
                                <th>SUMMARY_LVL</th>
                                <th>SUMMARY_LVL2</th>
                            </tr>
                        </thead>
                        <tbody>
                            <tr>
                                <td>1</td>
                                <td>MINERAL FUEL, OIL ETC.; BITUMIN SUBST; MINERAL WAX</td>
                                <td>HS2</td>
                                <td>4</td>
                                <td>BOSTON, MA</td>
                                <td>3010</td>
                                <td>COLOMBIA</td>
                                <td>6658994</td>
                                <td>6658994</td>
                                <td>2013</td>
                                <td>10</td>
                                <td>27</td>
                                <td>DET</td>
                                <td>HSCYDT</td>
                            </tr>
                        </tbody>
                    </table> </br></br>
                    The value for SUMMARY_LVL2 equals 'HSCYDT', which means for all goods that fall under 'HS' code (27 specified in I_COMMODITY), which were imported from Country 'CY' into district 'DT', the general imports valuation (in USD) for the 10th month in 2013 was $6658994 and Comsumption value was $6658994.</br></br>
            </td>
        </tr>
        <tr>
            <td><strong>YEAR</strong></td>
            <td>We'll pull the data from 2013 to 2023, these are the years for which monthly data is present </br></br></td>
        </tr>
        <tr>
            <td><strong>MONTH</strong></td>
            <td>We'll include all the months </br></br></td>
        </tr>
    </tbody>
</table>

</body>
</html>


In [11]:
# Fetching Monthly Import Data (from 2013 to 2023)

VARIABLES = [
            # 'I_COMMODITY',
            'I_COMMODITY_SDESC',
            'COMM_LVL', 
            'DISTRICT',
            'DIST_NAME',
            'CTY_CODE',
            'CTY_NAME',
            'GEN_VAL_MO', 
            'CON_VAL_MO'
            ]
PREDICATES = {}
PREDICATES['get'] = ",".join(VARIABLES)
PREDICATES['YEAR'] = ['20' + str(i+1) for i in range(12,23)]
PREDICATES['MONTH'] = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'] #[str(i+1) for i in range(12)]
PREDICATES['I_COMMODITY'] = '27*'
PREDICATES['SUMMARY_LVL'] = 'DET'
PREDICATES['SUMMARY_LVL2'] = 'HSCYDT'


ImportsData, _ = dr.retrieve_data(url= importsUrl, predicates=PREDICATES)
dh.to_parquet(ImportsData, 'ImportsData.parquet')
del ImportsData
ImDT = dh.from_parquet('ImportsData.parquet')


In [12]:
ImDT

,I_COMMODITY_SDESC,COMM_LVL,DISTRICT,DIST_NAME,CTY_CODE,CTY_NAME,GEN_VAL_MO,CON_VAL_MO,YEAR,MONTH,I_COMMODITY,SUMMARY_LVL,SUMMARY_LVL2
0,"MINERAL FUEL, OIL ETC.; BITUMIN SUBST; MINERAL...",HS2,20,"NEW ORLEANS, LA",2740,TRINIDAD AND TOBAGO,34650,34650,2013,01,27,DET,HSCYDT
1,"MINERAL FUEL, OIL ETC.; BITUMIN SUBST; MINERAL...",HS2,21,"PORT ARTHUR, TX",2740,TRINIDAD AND TOBAGO,2379132,2379132,2013,01,27,DET,HSCYDT
2,"MINERAL FUEL, OIL ETC.; BITUMIN SUBST; MINERAL...",HS2,32,"HONOLULU, HI",2740,TRINIDAD AND TOBAGO,1809294,1809294,2013,01,27,DET,HSCYDT
3,"MINERAL FUEL, OIL ETC.; BITUMIN SUBST; MINERAL...",HS2,49,"SAN JUAN, PR",2740,TRINIDAD AND TOBAGO,42626596,42626596,2013,01,27,DET,HSCYDT
4,"MINERAL FUEL, OIL ETC.; BITUMIN SUBST; MINERAL...",HS2,53,"HOUSTON-GALVESTON, TX",2740,TRINIDAD AND TOBAGO,75433025,75433025,2013,01,27,DET,HSCYDT
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1009548,ELECTRICAL ENERGY,HS10,07,"OGDENSBURG, NY",1220,CANADA,84228921,84228921,2023,12,2716000000,DET,HSCYDT
1009549,ELECTRICAL ENERGY,HS10,09,"BUFFALO, NY",1220,CANADA,42256450,42256450,2023,12,2716000000,DET,HSCYDT
1009550,ELECTRICAL ENERGY,HS10,30,"SEATTLE, WA",1220,CANADA,62186284,62186284,2023,12,2716000000,DET,HSCYDT
1009551,ELECTRICAL ENERGY,HS10,33,"GREAT FALLS, MT",1220,CANADA,5929084,5929084,2023,12,2716000000,DET,HSCYDT


<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8">
<meta name="viewport" content="width=device-width, initial-scale=1.0">
<title>Variables for retrieval of imports data</title>
<style>
    table {
        width: 90%;
        border-collapse: collapse;
    }
    th, td {
        padding: 8px;
        text-align: left;
        border-bottom: 1px solid #ddd;
    }
    th {
        background-color: #f2f2f2;
    }
        td {
    max-width: 400px; /* Adjust the maximum width as needed */
    overflow: auto; /* Add a scrollbar for overflow */
}
</style>
</head>
<body>

<h2>Variables for retrieval of export data</h2>
<h3>Helpful to understand the data we're trying to pull</h3>

<table>
    <thead>
        <tr>
            <th>Variable Name</th>
            <th>Context</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td><strong>E_COMMODITY</strong></td>
            <td>
                <p>Export Commodity or HS Code in the following case (Could be 2, 4, 6 or 10 character depending on Commodity Level)</p>
                <ul>
                    <li>
                        If it is 2 characters, it determines the chapter of commodity, which is a broad classification
                            <strong><em>for example code 27 includes includes everything in MINERAL FUEL, OIL ETC.; BITUMIN SUBST; MINERAL WAX </em></strong>
                    </li> </br>
                    <li>
                        If it is 4 characters, than it is further classification of all goods in chapter 27, denoted by first 2 characters and headings in the next 2 characters 
                            <strong><em>for example code 2701 includes includes everything in COAL; BRIQUETTES, OVOIDS ETC. MFR FROM COAL, which is first heading of chapter 27 </em></strong>
                    </li></br>
                    <li>
                        If it is 6 characters, than it is further classification of all goods in chapter 27, denoted by first 2 characters and a specific heading in the following 2 characters and a sub heading of the commodity in the last two characters
                        <strong><em>for example code 270111 includes includes everything in ANTHRACITE COAL, NOT AGGLOMERATED,  which is eleventh subheading of first heading of chapter 27</em></strong>
                    </li></br>
                    <li>
                        If it is 10 characters, than it is the exact identity or code of the product being imported
                        <strong><em>for example code 2701110010 is the product METALLURGICAL BITUMINOUS COAL NOT AGGLOMERATED, which is tenth product in eleventh subheading in the first heading of chapter 27 </em></strong>
                    </li></br>
                    <li>
                        To understand more in depth, please refer the following <a href="https://www.freightos.com/freight-resources/harmonized-system-code-finder-hs-code-lookup/">link</a> for easy explanation
                    </li>
                </ul></br></br>
            </td>
        </tr>
        <tr>
            <td><strong>E_COMMODITY_SDESC</strong></td>
            <td>
                <p>A short description of the Commodity Codes</p>
                <ul>
                    <li> <strong><em> 27 </em></strong> , comes under  <strong><em> HS2 COMM_LVL: </em></strong> with short description as MINERAL FUEL, OIL ETC.; BITUMIN SUBST; MINERAL WAX</li>
                    <li> <strong><em> 2701 </em></strong> , comes under  <strong><em> HS4 COMM_LVL: </em></strong> with short description as COAL; BRIQUETTES, OVOIDS ETC. MFR FROM COAL</li>
                    <li> <strong><em> 270111 </em></strong> , comes under  <strong><em> HS6 COMM_LVL: </em></strong> with short description as ANTHRACITE COAL, NOT AGGLOMERATED</li>
                    <li> <strong><em> 2701120010 </em></strong> , comes under  <strong><em> HS10 COMM_LVL: </em></strong> with short description as METALLURGICAL BITUMINOUS COAL NOT AGGLOMERATED</li>
                </ul></br></br>
            </td>
        </tr>
        <tr>
            <td><strong>COMM_LVL</strong></td>
            <td>
                <p>Indicates whether a row contains information of a commodity on broad level or detailed level (HS2, HS4, HS6 or HS10)</p></br></br>
            </td>
        </tr>
        <tr>
            <td><strong>DISTRICT</strong></td>
            <td>Code of the district from where the goods were exported</br></br></td>
        </tr>
        <tr>
            <td><strong>DIST_NAME</strong></td>
            <td>Name of the district from where the goods were exported</br></br></td>
        </tr>
        <tr>
            <td><strong>CTY_CODE</strong></td>
            <td>Code of the country to which goods were exported from the states</br></br></td>
        </tr>
        <tr>
            <td><strong>CTY_NAME</strong></td>
            <td>Name of the country to which goods were exported from the states</br></br></td>
        </tr>
        <tr>
            <td><strong>ALL_VAL_MO</strong></td>
            <td>Total Valuation of goods exported</br></br></td>
        </tr>
        <tr>
            <td><strong>QTY_1_MO</strong></td>
            <td rowspan="6"><strong>To understand Quantities and Flags, please refer to the following <a href="https://www.census.gov/foreign-trade/reference/definitions/index.html">link</a> and this <a href="https://www.census.gov/foreign-trade/reference/guides/Guide_to_International_Trade_Datasets.pdf">pdf</a></strong></td>
        </tr>
        <tr>
            <td><strong>QTY_1_MO_FLAG</strong></td>
        </tr>
        <tr>
            <td><strong>QTY_2_MO</strong></td>
        </tr>
        <tr>
            <td><strong>QTY_2_MO_FLAG</strong></td>
        </tr>
        <tr>
            <td><strong>UNITQY1</strong></td>
        </tr>
        <tr>
            <td><strong>UNITQY2</strong></td>
        </tr>
        <tr>
            <td><strong>SUMMARY_LVL</strong></td>
            <td>
                SUMMARY_LVL has only two potential values: CGP and DET. We choose DET for our Analysis. CGP represents data aggregated by different country groupings, such as geographic regions (e.g. Africa), international organizations (e.g. European Union), or free trade agreements (e.g. NAFTA). DET represents individual country data not summarized by any country groupings. The SUMMARY_LVL parameter is useful if you want to only show trade data by individual countries in your application (using DET) or if you want to exclude individual countries and only show trade by country groupings (using CGP).</br></br>
            </td>
        </tr>
        <tr>
            <td><strong>SUMMARY_LVL2</strong></td>
            <td>
                The second parameter that accounts for different levels of data summarization is SUMMARY_LVL2. SUMMARY_LVL2 allows you to view and pull data from the API based on which variables from the trade datasets are combined and summarized (Kind of pre-performed Group By Tasks like we do in Pandas) </br></br></br>
                For Example, if a row contains values as follows: </br></br>
                    <table border="1">
                        <thead>
                            <tr>
                                <th></th>
                                <th>E_COMMODITY_SDESC</th>
                                <th>COMM_LVL</th>
                                <th>DISTRICT</th>
                                <th>DIST_NAME</th>
                                <th>CTY_CODE</th>
                                <th>CTY_NAME</th>
                                <th>ALL_VAL_MO</th>
                                <th>QTY_1_MO</th>
                                <th>QTY_1_MO_FLAG</th>
                                <th>QTY_2_MO</th>
                                <th>QTY_2_MO_FLAG</th>
                                <th>UNIT_QY1</th>
                                <th>UNIT_QY2</th>
                                <th>YEAR</th>
                                <th>MONTH</th>
                                <th>E_COMMODITY</th>
                                <th>SUMMARY_LVL</th>
                                <th>SUMMARY_LVL2</th>
                            </tr>
                        </thead>
                        <tbody>
                            <tr>
                                <td rowspan="6"><strong>To understand Quantities and Flags, please refer to the following link</strong></td>
                                <td>MINERAL FUEL, OIL ETC.; BITUMIN SUBST; MINERAL WAX</td>
                                <td>HS2</td>
                                <td>11</td>
                                <td>PHILADELPHIA, PA</td>
                                <td>2010</td>
                                <td>MEXICO</td>
                                <td>0</td>
                                <td>0</td>
                                <td>M</td>
                                <td>0</td>
                                <td>M</td>
                                <td>-</td>
                                <td>-</td>
                                <td>2013</td>
                                <td>10</td>
                                <td>27</td>
                                <td>DET</td>
                                <td>HSDTCY</td>
                            </tr>
                        </tbody>
                    </table>
                    The value for SUMMARY_LVL2 equals 'HSDTCY', which means for all goods that fall under 'HS' code (27 specified in I_COMMODITY), which were exported from district 'DT' to the Country 'CY', the exportss valuation (in USD) for the 10th month in 2013 was $0 and Comsumption value was $0 from Philadelphia to Mexico.</br></br>
            </td>
        </tr>
        <tr>
            <td><strong>YEAR</strong></td>
            <td>We'll pull the data from 2013 to 2023, these are the years for which monthly data is present </br></br></td>
        </tr>
        <tr>
            <td><strong>MONTH</strong></td>
            <td>We'll include all the months </br></br></td>
        </tr>
    </tbody>
</table>

</body>
</html>


In [13]:
# Fetching Export Data

VARIABLES = [
            'E_COMMODITY_SDESC',
            'COMM_LVL', 
            'DISTRICT',
            'DIST_NAME',
            'CTY_CODE',
            'CTY_NAME',
            'ALL_VAL_MO'
            ]
PREDICATES = {}

PREDICATES['YEAR'] = ['20' + str(i+1) for i in range(12,23)]
PREDICATES['MONTH'] = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
PREDICATES['E_COMMODITY'] = '27*'
PREDICATES['SUMMARY_LVL'] = 'DET'
PREDICATES['SUMMARY_LVL2'] = 'HSDTCY'
PREDICATES['get'] = ",".join(VARIABLES)

# PREDICATES['key'] = API


ExportsData, _ = dr.retrieve_data(url= exportsUrl, predicates=PREDICATES)
dh.to_parquet(ExportsData, 'ExportsData.parquet')
del ExportsData
ExDT = dh.from_parquet('ExportsData.parquet')

